<a href="https://colab.research.google.com/github/lucaskrieger24/PML_exam/blob/main/BERT_esm1b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

install and load models

In [1]:
%%time
# for other models see: https://github.com/facebookresearch/esm#main-models-you-should-use-

# mount google-drive, download to google-drive,
# will run faster next time!
'''
import os
import torch
hub_dir = "/content/drive/MyDrive/AlgoSB"
if not os.path.isdir(hub_dir): os.mkdir(hub_dir)
torch.hub.set_dir(hub_dir)
model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm1b_t33_650M_UR50S")
'''

# this step will take ~3mins
import torch
import os
os.system("apt-get install aria2 -qq")
os.system("mkdir -p /root/.cache/torch/hub/checkpoints/")
# os.system("aria2c -q -x 16 -d /root/.cache/torch/hub/ https://github.com/facebookresearch/esm/archive/main.zip")
# os.system("aria2c -q -x 16 -d /root/.cache/torch/hub/checkpoints/ https://dl.fbaipublicfiles.com/fair-esm/models/esm1b_t33_650M_UR50S.pt")
# os.system("aria2c -q -x 16 -d /root/.cache/torch/hub/checkpoints/ https://dl.fbaipublicfiles.com/fair-esm/regression/esm1b_t33_650M_UR50S-contact-regression.pt")
# # model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm1b_t33_650M_UR50S")

CPU times: user 1.25 s, sys: 252 ms, total: 1.51 s
Wall time: 4.98 s


0

In [2]:
model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm2_t33_650M_UR50D")

Using cache found in /root/.cache/torch/hub/facebookresearch_esm_main


In [3]:
# put model on GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model = model.eval()
#model.args.token_dropout = False

# alphabet size
esm_alphabet_len = len(alphabet.all_toks)
esm_alphabet = "".join(alphabet.all_toks[4:24])+"-"

In [4]:
!wget -qnc https://raw.githubusercontent.com/sokrypton/algosb_2021/main/utils.py
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import softmax
from utils import *

In [5]:
protein_seq = "MANINLKEITLIVGVVTACYWNSLFCGFVFDDVSAILDNKDLHPSTPLKTLFQNDFWGTPMSEERSHKSYRPLTVLTFRLNYLLSELKPMSYHLLNMIFHAVVSVIFLKVCKLFLDNKSSVIASLLFAVHPIHTEAVTGVVGRAELLSSIFFLAAFLSYTRSKGPDNSIIWTPIALTVFLVAVATLCKEQGITVVGICCVYEVFIAQGYTLPLLCTTAGQFLRGKGSIPFSMLQTLVKLIVLMFSTLLLVVIRVQVIQSQLPVFTRFDNPAAVSPTPTRQLTFNYLLPVNAWLLLNPSELCCDWTMGTIPLIESLLDIRNLATFTFFCFLGMLGVFSIRYSGDSSKTVLMALCLMALPFIPASNLFFPVGFVVAERVLYVPSMGFCILVAHGWQKISTKSVFKKLSWICLSMVILTHSLKTFHRNWDWESEYTLFMSALKVNKNNAKLWNNVGHALENEKNFERALKYFLQATHVQPDDIGAHMNVGRTYKNLNRTKEAEESYMMAKSLMPQIIPGKKYAARIAPNHLNVYINLANLIRANESRLEEADQLYRQAISMRPDFKQAYISRGELLLKMNKPLKAKEAYLKALELDRNNADLWYNLAIVHIELKEPNEALKKNFNRALELNPKHKLALFNSAIVMQESGEVKLRPEARKRLLSYINEEPLDANGYFNLGMLAMDDKKDNEAEIWMKKAIKLQADFRSALFNLALLYSQTAKELKALPILEELLRYYPDHIKGLILKGDILMNQKKDILGAKKCFERILEMDPSNVQGKHNLCVVYFEEKDLLKAERCLLETLALAPHEEYIQRHLNIVRDKISSSSFIEPIFPTSKISSVEGKKIPTESVKEIRGESRQTQIVKTSDNKSQSKSNKQLGKNGDEETPHKTTKDIKEIEKKRVAALKRLEEIERILNGE"
linker = "GSGS"
binder = "MEIDEPLTRRAMPSS"
seq = linker + protein_seq + binder

In [6]:
def get_logits(seq):
  x,ln = alphabet.get_batch_converter()([("seq",seq)])[-1],len(seq)
  with torch.no_grad():
    f = lambda x: model(x)["logits"][0,1:(ln+1),4:24].cpu().numpy()
    x = x.to(device)
    logits = f(x)
    return logits

def get_masked_logits(seq, p=None, get_pll=False):
  x,ln = alphabet.get_batch_converter()([(None,seq)])[-1],len(seq)
  if p is None: p = ln
  with torch.no_grad():
    def f(x):
      fx = model(x)["logits"][:,1:(ln+1),4:24]
      return fx

    logits = np.zeros((ln,20))
    for n in range(0,ln,p):
      m = min(n+p,ln)
      x_h = torch.tile(torch.clone(x),[m-n,1])
      for i in range(m-n):
        x_h[i,n+i+1] = alphabet.mask_idx
      fx_h = f(x_h.to(device))
      for i in range(m-n):
        logits[n+i] = fx_h[i,n+i].cpu().numpy()
  if get_pll:
    logits = np.log(softmax(logits,-1))
    x = x.cpu().numpy()[0]
    x = x[1:(ln+1)] - 4
    return sum([logits[n,i] for n,i in enumerate(x)])
  else:
    return logits

In [7]:
def get_categorical_jacobian(seq):
  # ∂in/∂out
  x,ln = alphabet.get_batch_converter()([("seq",seq)])[-1],len(seq)
  with torch.no_grad():
    f = lambda x: model(x)["logits"][...,1:(ln+1),4:24].cpu().numpy()
    fx = f(x.to(device))[0]
    x = torch.tile(x,[20,1]).to(device)
    fx_h = np.zeros((ln,20,ln,20))
    for n in range(ln): # for each position
      x_h = torch.clone(x)
      x_h[:,n+1] = torch.arange(4,24) # mutate to all 20 aa
      fx_h[n] = f(x_h)
    return fx-fx_h

In [ ]:
# jacobian of the model
jac = get_categorical_jacobian(seq)
# center & symmetrize
for i in range(4): jac -= jac.mean(i,keepdims=True)
jac = (jac + jac.transpose(2,3,0,1))/2

extract contacts

In [ ]:
plt.figure(figsize=(10,5))
#plt.subplot(1,2,1); plt.title("inv_cov(msa)"); plt.imshow(tmp["apc"])
plt.subplot(1,2,2); plt.title("jac(esm2(msa[0]))"); plt.imshow(get_contacts(jac))
plt.show()

In [ ]:
i = 9
j = 12

# plt.figure(figsize=(10,5))
# plt.subplot(1,2,1);
# plt.title("inv_cov(msa)");
# plt.imshow(ic[i,:,j,:],vmin=-1.5,vmax=1.5,cmap="bwr_r")
# plt.xticks(range(20),esm_alphabet[:20])
# plt.yticks(range(20),esm_alphabet[:20])

plt.subplot(1,2,2);
plt.title("jac(esm1b(msa[0]))");
plt.imshow(jac[i,:,j,:],vmin=-3,vmax=3,cmap="bwr_r")
plt.xticks(range(20),esm_alphabet[:20])
plt.yticks(range(20),esm_alphabet[:20])
plt.show()